In [1]:
import sys
import os
from huggingface_hub import HfApi
from datetime import datetime
import pandas as pd
import re
from openai import OpenAI

parent_dir = os.path.dirname(os.getcwd())  # project root
sys.path.insert(0, parent_dir)


c:\Users\ssg_h\anaconda3\envs\genegpt_env_39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def check_model_supported(models, api_key=None, max_tokens=512):
    """
    Test if HuggingFace model(s) are supported using the same method as call_langchain_model.
    
    Args:
        models (str or list): Single model name or list of model names to test
        api_key (str, optional): HuggingFace API key. If None, uses HUGGINGFACE_API_KEY env var
    
    Returns:
        dict: Summary of results with same format regardless of input type
    """
    # Convert single string to list
    if isinstance(models, str):
        model_list = [models]
    else:
        model_list = models
    
    # Get API key
    if api_key is None:
        api_key = os.environ.get('HUGGINGFACE_API_KEY')
    
    if not api_key:
        return {
            "total_tested": len(model_list),
            "supported_count": 0,
            "correct_count": 0,
            "supported_models": [],
            "correct_models": [],
            "detailed_results": {model: {
                "supported": False,
                "response": "",
                "correct_answer": False,
                "error": "No API key provided"
            } for model in model_list}
        }
    
    # Import LangChain components (same as your actual code)
    try:
        from langchain_openai import ChatOpenAI
        from langchain_core.messages import SystemMessage, HumanMessage
    except ImportError:
        return {
            "total_tested": len(model_list),
            "supported_count": 0,
            "correct_count": 0,
            "supported_models": [],
            "correct_models": [],
            "detailed_results": {model: {
                "supported": False,
                "response": "",
                "correct_answer": False,
                "error": "LangChain imports not available"
            } for model in model_list}
        }
    
    results = {}
    supported_models = []
    correct_models = []
    
    # Define stop sequences (same as your actual code)
    STOP_SEQUENCE = ['->', '\n\nQuestion']
    
    print(f"Testing {len(model_list)} HuggingFace model(s)...")
    print("=" * 50)
    
    for model_name in model_list:
        if model_name.startswith("huggingface/"):
            model_name = model_name.replace("huggingface/", "")
        print(f"Testing: {model_name}")
        
        try:
            # Create ChatOpenAI model (same as your actual implementation)
            llm = ChatOpenAI(
                model=model_name,
                temperature=0.0,
                openai_api_key=api_key,
                openai_api_base="https://router.huggingface.co/v1",
                max_tokens=max_tokens
            )
            
            # Create messages (same format as call_langchain_model)
            system_message = "You are a calculator. Only output numbers, never show your thinking process."
            q_prompt = "You are a calculator. Only output numbers, never show your thinking process. What is 1+1? (just give me the answer)"
            
            messages = [
                SystemMessage(content=system_message),
                HumanMessage(content=q_prompt)
            ]
            
            # Call model (same as call_langchain_model)
            response = llm.invoke(messages, stop=STOP_SEQUENCE)
            response_text = response.content
            
            # Handle None response
            if response_text is None:
                response_text = ""
            else:
                response_text = response_text.strip()
            
            # For models that use thinking tags, try to extract the final answer
            final_answer = response_text
            if "<think>" in response_text:
                # Try to get text after </think> tag
                if "</think>" in response_text:
                    final_answer = response_text.split("</think>")[-1].strip()
                else:
                    # If thinking tag is not closed, this model might not be suitable
                    # But let's still check the original response
                    pass
            
            # Check if the answer is correct using the final answer
            response_lower = final_answer.lower()
            
            # Multiple ways to check for correct answer
            correct_answer = (
                "2" in final_answer or                     # Direct "2"
                "two" in response_lower or                 # Written as "two" 
                final_answer.strip() == "2" or             # Exact match
                final_answer.strip().endswith("2") or      # Ends with 2
                "1+1=2" in final_answer.replace(" ", "") or  # Shows equation
                "answer is 2" in response_lower or         # "The answer is 2"
                "result is 2" in response_lower            # "The result is 2"
            )
            
            result = {
                "supported": True,
                "response": response_text,
                "correct_answer": correct_answer,
                "error": None
            }
            
        except Exception as e:
            error_msg = str(e)
            
            # Parse common error types
            if "not supported" in error_msg.lower():
                error_type = "Model not supported"
            elif "402" in error_msg or "credits" in error_msg.lower():
                error_type = "Credits exhausted"
            elif "401" in error_msg or "unauthorized" in error_msg.lower():
                error_type = "Invalid API key"
            elif "429" in error_msg or "rate limit" in error_msg.lower():
                error_type = "Rate limited"
            else:
                error_type = "Unknown error"
            
            result = {
                "supported": False,
                "response": "",
                "correct_answer": False,
                "error": f"{error_type}: {error_msg}"
            }
        
        # Store result
        results[model_name] = result
        
        # Update counters and lists
        if result["supported"]:
            supported_models.append(model_name)
            status = f"✅ SUPPORTED - Response: '{result['response']}'"
            if result["correct_answer"]:
                correct_models.append(model_name)
                status += " (CORRECT ✓)"
            else:
                status += " (WRONG ✗)"
        else:
            status = f"❌ FAILED - {result['error']}"
        
        print(f"   {status}")
        print()
    
    # Print summary
    print("=" * 50)
    print(f"Summary:")
    print(f"  Total tested: {len(model_list)}")
    print(f"  Supported: {len(supported_models)}")
    print(f"  Correct answers: {len(correct_models)}")
    
    if correct_models:
        print(f"\n✅ Working models:")
        for model in correct_models:
            print(f"  - {model}")
    
    return {
        "total_tested": len(model_list),
        "supported_count": len(supported_models),
        "correct_count": len(correct_models),
        "supported_models": supported_models,
        "correct_models": correct_models,
        "detailed_results": results
    }


In [15]:
# analyze_author_models
def extract_parameters(model_name):
    """Extract number of parameters from model name and convert to integer"""
    # Look for patterns like 0.5B, 1.5B, 3B, 7B, 14B, 32B, 72B, 110B
    pattern = r'(\d+(?:\.\d+)?)[Bb]'
    match = re.search(pattern, model_name)
    
    if match:
        size_str = match.group(1)
        size_float = float(size_str)
        # Convert to actual number (B = billion)
        return int(size_float * 1_000_000_000)
    
    return None  # Unknown size

def is_instruct_model(model_info):
    instruct_keywords = {"instruct", "chat", "conversation", "conversational", "instruction-tuned"}
    # check tags first
    if model_info.tags and any(tag.lower() in instruct_keywords for tag in model_info.tags):
        return True
    # fallback: check ID if tags are missing
    return any(keyword in model_info.modelId.lower() for keyword in instruct_keywords)


def analyze_author_models(author_name, top_n=100, min_model_downloads=1):
    """Analyze models for a specific author"""
    api = HfApi()
    
    print(f"Fetching {author_name} text-generation models...")
    
    try:
        models = api.list_models(
            author=author_name, 
            # task="text-generation",
            limit=None,
            sort="lastModified",
            direction=-1
        )
        
        # Filter for likely chat/instruct models
        good_models = []
        
        for model in models:
            # Look for instruct/chat indicators
            #is_instruct = any(keyword in model.id.lower() for keyword in [
            #    'instruct', 'chat', 'conversation', 'conversational', 'text-generation-inference'
            #])
            is_instruct = is_instruct_model(model)
            print(f"Checking {model.id}: is_instruct={is_instruct}, downloads={model.downloads}, private={model.private}")
            # Check if it has good stats and is public
            if is_instruct and not model.private and model.downloads > min_model_downloads:
                good_models.append({
                    'id': model.id,
                    'created_at': model.created_at,
                    'downloads': model.downloads,
                    'likes': model.likes,
                    'tags': model.tags
                })
        
        # Sort by creation date (newest first)
        good_models.sort(key=lambda x: x['created_at'] if x['created_at'] else datetime.min, reverse=True)
        
        # Take top N models
        top_models = good_models[:top_n]
        
        if not top_models:
            print(f"No suitable models found for {author_name}")
            return pd.DataFrame()
        
        print(f"Found {len(top_models)} {author_name} models to analyze")
        
        # Test all models for support
        model_ids_for_testing = [model['id'] for model in top_models]
        print(f"Testing {len(model_ids_for_testing)} models for HuggingFace support...")
        
        # Test the models
        support_result = check_model_supported(model_ids_for_testing)
        
        # Create DataFrame
        df_data = []
        
        for model in top_models:
            model_name = model['id']
            
            # Extract just the model name (remove author prefix)
            display_name = model_name.split('/')[-1]
            
            # Create URL
            url = f"https://huggingface.co/{model_name}"
            
            # Extract parameters
            parameters = extract_parameters(model_name)
            
            # Format release date
            released = model['created_at'].strftime('%Y-%m-%d') if model['created_at'] else None
            
            # Downloads
            downloads = model['downloads']
            
            # Check if supported
            supported = model_name in support_result['correct_models']
            
            df_data.append({
                'author': author_name,
                'model_name': display_name,
                'url': url,
                'parameters': parameters,
                'released': released,
                'downloads': downloads,
                'supported': supported
            })
        
        # Create DataFrame
        df = pd.DataFrame(df_data)
        
        # Sort by release date (newest first) and then by downloads
        df['released_dt'] = pd.to_datetime(df['released'])
        df = df.sort_values(['released_dt', 'downloads'], ascending=[False, False])
        
        # Drop the helper column
        df = df.drop('released_dt', axis=1)
        
        print(f"{author_name} Analysis Summary:")
        print(f"  Total models: {len(df)}")
        print(f"  Supported models: {df['supported'].sum()}")
        print(f"  Models with known parameters: {df['parameters'].notna().sum()}")
        
        return df
        
    except Exception as e:
        print(f"Error analyzing {author_name}: {e}")
        return pd.DataFrame()


In [23]:
from huggingface_hub import HfApi

def get_model_tasks(model_name: str):
    api = HfApi()
    model_info = api.model_info(model_name)
    return {
        "primary_task": model_info.pipeline_tag,
        "all_tags": model_info.tags
    }

# Example usage
info = get_model_tasks("google/gemma-3-27b-it")
print(info)



{'primary_task': 'image-text-to-text', 'all_tags': ['transformers', 'safetensors', 'gemma3', 'image-text-to-text', 'conversational', 'arxiv:1905.07830', 'arxiv:1905.10044', 'arxiv:1911.11641', 'arxiv:1904.09728', 'arxiv:1705.03551', 'arxiv:1911.01547', 'arxiv:1907.10641', 'arxiv:1903.00161', 'arxiv:2009.03300', 'arxiv:2304.06364', 'arxiv:2103.03874', 'arxiv:2110.14168', 'arxiv:2311.12022', 'arxiv:2108.07732', 'arxiv:2107.03374', 'arxiv:2210.03057', 'arxiv:2106.03193', 'arxiv:1910.11856', 'arxiv:2502.12404', 'arxiv:2502.21228', 'arxiv:2404.16816', 'arxiv:2104.12756', 'arxiv:2311.16502', 'arxiv:2203.10244', 'arxiv:2404.12390', 'arxiv:1810.12440', 'arxiv:1908.02660', 'arxiv:2312.11805', 'base_model:google/gemma-3-27b-pt', 'base_model:finetune:google/gemma-3-27b-pt', 'license:gemma', 'text-generation-inference', 'endpoints_compatible', 'region:us']}


In [ ]:
model_to_test = [
    "huggingface/zai-org/GLM-4.5",
    "huggingface/zai-org/GLM-4.5-Air", 
    "huggingface/nvidia/Llama-3.1-Nemotron-Nano-8B-v1", 
    "huggingface/meta-llama/Llama-3.1-8B-Instruct", 
    "huggingface/openai/gpt-oss-120b", 
    "huggingface/openai/gpt-oss-20b", 
    "huggingface/nvidia/Mistral-NeMo-Minitron-8B-Instruct", 
    "huggingface/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B", 
    "huggingface/tiiuae/falcon-7b-instruct", 
    "huggingface/epfl-llm/meditron-70b", 
    "huggingface/epfl-llm/meditron-7b",
    "huggingface/Qwen/Qwen2.5-Coder-32B-Instruct",
    "google/gemma-3-27b-it",
    "huggingface/Qwen/Qwen2.5-Coder-7B-Instruct",
    "huggingface/Qwen/Qwen2.5-Coder-3B-Instruct",
    "HuggingFaceTB/SmolLM3-3B",
    "Qwen/Qwen2.5-72B-Instruct"
]
model_to_test = ["huggingface/Qwen/Qwen2.5-Coder-32B-Instruct",
                 "huggingface/Qwen/Qwen2.5-Coder-14B-Instruct",
                 "huggingface/Qwen/Qwen2.5-Coder-7B-Instruct",
                 "huggingface/Qwen/Qwen2.5-Coder-3B-Instruct",
                 "huggingface/Qwen/Qwen2.5-Coder-1.5B-Instruct",
                 "huggingface/Qwen/Qwen2.5-Coder-0.5B-Instruct",
                 "huggingface/Qwen/Qwen2.5-72B-Instruct"
]
import pandas as pd

# Run test
result = check_model_supported(model_to_test, max_tokens=100)

# Convert to DataFrame
df = pd.DataFrame.from_dict(result['detailed_results'], orient='index')

# Display the table
display(df)

# Or with better formatting
df.style.set_properties(**{'text-align': 'left'})


Testing 7 HuggingFace model(s)...
Testing: Qwen/Qwen2.5-Coder-32B-Instruct
   ✅ SUPPORTED - Response: '2' (CORRECT ✓)

Testing: Qwen/Qwen2.5-Coder-14B-Instruct
   ❌ FAILED - Model not supported: Error code: 400 - {'error': {'message': "The requested model 'Qwen/Qwen2.5-Coder-14B-Instruct' is not supported by any provider you have enabled.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_supported'}}

Testing: Qwen/Qwen2.5-Coder-7B-Instruct
   ✅ SUPPORTED - Response: '2' (CORRECT ✓)

Testing: Qwen/Qwen2.5-Coder-3B-Instruct
   ✅ SUPPORTED - Response: '2' (CORRECT ✓)

Testing: Qwen/Qwen2.5-Coder-1.5B-Instruct
   ❌ FAILED - Model not supported: Error code: 400 - {'error': {'message': "The requested model 'Qwen/Qwen2.5-Coder-1.5B-Instruct' is not supported by any provider you have enabled.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_supported'}}

Testing: Qwen/Qwen2.5-Coder-0.5B-Instruct
   ❌ FAILED - Model not supported: Error code: 400 - 

,supported,response,correct_answer,error
Qwen/Qwen2.5-Coder-32B-Instruct,True,2,True,None
Qwen/Qwen2.5-Coder-14B-Instruct,False,,False,Model not supported: Error code: 400 - {'error...
Qwen/Qwen2.5-Coder-7B-Instruct,True,2,True,None
Qwen/Qwen2.5-Coder-3B-Instruct,True,2,True,None
Qwen/Qwen2.5-Coder-1.5B-Instruct,False,,False,Model not supported: Error code: 400 - {'error...
Qwen/Qwen2.5-Coder-0.5B-Instruct,False,,False,Model not supported: Error code: 400 - {'error...
Qwen/Qwen2.5-72B-Instruct,True,2,True,None


,supported,response,correct_answer,error
Qwen/Qwen2.5-Coder-32B-Instruct,True,2,True,None
Qwen/Qwen2.5-Coder-14B-Instruct,False,,False,"Model not supported: Error code: 400 - {'error': {'message': ""The requested model 'Qwen/Qwen2.5-Coder-14B-Instruct' is not supported by any provider you have enabled."", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_supported'}}"
Qwen/Qwen2.5-Coder-7B-Instruct,True,2,True,None
Qwen/Qwen2.5-Coder-3B-Instruct,True,2,True,None
Qwen/Qwen2.5-Coder-1.5B-Instruct,False,,False,"Model not supported: Error code: 400 - {'error': {'message': ""The requested model 'Qwen/Qwen2.5-Coder-1.5B-Instruct' is not supported by any provider you have enabled."", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_supported'}}"
Qwen/Qwen2.5-Coder-0.5B-Instruct,False,,False,"Model not supported: Error code: 400 - {'error': {'message': ""The requested model 'Qwen/Qwen2.5-Coder-0.5B-Instruct' is not supported by any provider you have enabled."", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_supported'}}"
Qwen/Qwen2.5-72B-Instruct,True,2,True,None


In [25]:

# Authors to analyze
authors = ["mistralai", "meta-llama", "google", "microsoft", "tiiuae"]
# authors = ["Qwen", "deepseek-ai", "zai-org"]
authors = ["HuggingFaceTB"]
print("=" * 80)

all_dataframes = []

for author in authors:
    print(f"\n=== Analyzing {author} ===")
    df_author = analyze_author_models(author, top_n=100)
    
    if not df_author.empty:
        all_dataframes.append(df_author)
        
        # Show summary for this author
        working_models = df_author[df_author['supported'] == True]
        print(f"\nWorking {author} models ({len(working_models)}):")
        for _, row in working_models.head(5).iterrows():  # Show top 5 working
            params_str = f"{row['parameters']/1_000_000_000:g}B" if pd.notna(row['parameters']) else "Unknown"
            print(f"  - {row['model_name']} ({params_str}, {row['downloads']:,} downloads)")
        
        if len(working_models) > 5:
            print(f"  ... and {len(working_models) - 5} more")
    
    print("-" * 50)

# Combine all dataframes
if all_dataframes:
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    
    print(f"\n=== COMBINED ANALYSIS ===")
    print(f"Total models analyzed: {len(combined_df)}")
    print(f"Total supported models: {combined_df['supported'].sum()}")
    
    # Summary by author
    print(f"\nSupport rate by author:")
    author_summary = combined_df.groupby('author').agg({
        'supported': ['count', 'sum', lambda x: f"{(x.sum()/len(x)*100):.1f}%"]
    }).round(1)
    author_summary.columns = ['Total_Models', 'Supported_Models', 'Support_Rate']
    print(author_summary)
    
    # Show top working models across all authors
    all_working = combined_df[combined_df['supported'] == True].sort_values('downloads', ascending=False)
    print(f"\nTop 10 working models (by downloads):")
    for _, row in all_working.head(10).iterrows():
        params_str = f"{row['parameters']/1_000_000_000:g}B" if pd.notna(row['parameters']) else "Unknown"
        print(f"  {row['author']}/{row['model_name']} ({params_str}, {row['downloads']:,} downloads)")
    
    # Display the combined dataframe
    print(f"\nCombined DataFrame:")
    display(combined_df)
    # combined_df = combined_df[combined_df['supported'] == True]
    combined_df.to_csv('models_analysis_' + "+".join(authors) + '.csv', index=False)
    # Return the combined dataframe
    combined_df
else:
    print("No data retrieved for any authors")


=== Analyzing HuggingFaceTB ===
Fetching HuggingFaceTB text-generation models...
Checking HuggingFaceTB/SmolLM3-3B-checkpoints: is_instruct=False, downloads=1666, private=False
Checking HuggingFaceTB/SmolLM3-3B-Base: is_instruct=False, downloads=9625, private=False
Checking HuggingFaceTB/SmolLM3-3B: is_instruct=True, downloads=586058, private=False
Checking HuggingFaceTB/smollm2-135M-SFT-Only: is_instruct=False, downloads=49, private=False
Checking HuggingFaceTB/SmolLM3-3B-ONNX: is_instruct=True, downloads=436, private=False
Checking HuggingFaceTB/SmolLM2-360M-Instruct: is_instruct=True, downloads=526226, private=False
Checking HuggingFaceTB/SmolLM2-135M-Instruct: is_instruct=True, downloads=390272, private=False
Checking HuggingFaceTB/SmolLM2-1.7B-Instruct: is_instruct=True, downloads=47671, private=False
Checking HuggingFaceTB/SmolVLM2-2.2B-Base: is_instruct=True, downloads=960, private=False
Checking HuggingFaceTB/SmolVLM-256M-Instruct: is_instruct=True, downloads=510242, private=F

,author,model_name,url,parameters,released,downloads,supported
0,HuggingFaceTB,SmolLM3-3B,https://huggingface.co/HuggingFaceTB/SmolLM3-3B,3.000000e+09,2025-07-08,586058,True
1,HuggingFaceTB,SmolLM3-3B-ONNX,https://huggingface.co/HuggingFaceTB/SmolLM3-3...,3.000000e+09,2025-07-08,436,False
2,HuggingFaceTB,SmolVLM2-2.2B-Base,https://huggingface.co/HuggingFaceTB/SmolVLM2-...,2.200000e+09,2025-04-14,960,False
3,HuggingFaceTB,SmolLM2-1.7B-Instruct-16k,https://huggingface.co/HuggingFaceTB/SmolLM2-1...,1.700000e+09,2025-02-21,7,False
4,HuggingFaceTB,SmolVLM2-256M-Video-Instruct,https://huggingface.co/HuggingFaceTB/SmolVLM2-...,NaN,2025-02-11,123138,False
5,HuggingFaceTB,SmolVLM2-500M-Video-Instruct,https://huggingface.co/HuggingFaceTB/SmolVLM2-...,NaN,2025-02-11,64055,False
6,HuggingFaceTB,SmolVLM2-2.2B-Instruct,https://huggingface.co/HuggingFaceTB/SmolVLM2-...,2.200000e+09,2025-02-08,122121,False
7,HuggingFaceTB,SmolVLM-500M-Instruct,https://huggingface.co/HuggingFaceTB/SmolVLM-5...,NaN,2025-01-20,38117,False
8,HuggingFaceTB,SmolVLM-256M-Instruct,https://huggingface.co/HuggingFaceTB/SmolVLM-2...,NaN,2025-01-17,510242,False
9,HuggingFaceTB,SmolLM2-135M-Instruct-Q8-mlx,https://huggingface.co/HuggingFaceTB/SmolLM2-1...,NaN,2024-11-27,37,False


In [37]:
from huggingface_hub import HfApi

# Get models for a specific provider
api = HfApi()

# Get models with inference provider filter
models = api.list_models(
    limit=None  # Get all models
)
'''
models = api.list_models(
    filter="inference_provider:fireworks-ai",
    limit=None  # Get all models
)
'''
# Extract just the model names
model_names = [model.id for model in models]

print(f"Found {len(model_names)} models for fireworks-ai:")
for name in model_names[:10]:  # Show first 10
    print(f"  - {name}")

Found 1963210 models for fireworks-ai:
  - Qwen/Qwen-Image-Edit
  - google/gemma-3-270m
  - tencent/Hunyuan-GameCraft-1.0
  - deepseek-ai/DeepSeek-V3.1-Base
  - openai/gpt-oss-20b
  - zai-org/GLM-4.5V
  - google/gemma-3-270m-it
  - Qwen/Qwen-Image
  - openai/gpt-oss-120b
  - janhq/Jan-v1-4B


In [6]:
from huggingface_hub import HfApi

api = HfApi()

# Get Qwen text-generation models specifically
qwen_text_models = api.list_models(
    author="Qwen",
    task="text-generation",
    limit=None,
    sort="downloads",
    direction=-1
)

qwen_text_list = list(qwen_text_models)
print(f"Found {len(qwen_text_list)} Qwen text-generation models:")

# Show with more details
for model in qwen_text_list:
    print(f"\n📦 {model.id}")
    print(f"   ⬇️  {model.downloads:,} downloads, ❤️  {model.likes} likes")
    print(f"   📅 Created: {model.created_at.strftime('%Y-%m-%d') if model.created_at else 'N/A'}")
    
    # Look for chat/instruct models
    is_instruct = any(keyword in model.id.lower() for keyword in ['instruct', 'chat'])
    if is_instruct:
        print(f"   🤖 Chat/Instruct model ✓")
    
    # Show relevant tags
    if model.tags:
        relevant_tags = [tag for tag in model.tags if 
                        any(keyword in tag.lower() for keyword in 
                            ['instruct', 'chat', 'inference', 'compatible'])]
        if relevant_tags:
            print(f"   🏷️  {relevant_tags}")

Found 294 Qwen text-generation models:

📦 Qwen/Qwen2.5-14B-Instruct
   ⬇️  12,195,236 downloads, ❤️  264 likes
   📅 Created: 2024-09-16
   🤖 Chat/Instruct model ✓
   🏷️  ['chat', 'autotrain_compatible', 'text-generation-inference', 'endpoints_compatible']

📦 Qwen/Qwen2.5-7B-Instruct
   ⬇️  9,128,290 downloads, ❤️  760 likes
   📅 Created: 2024-09-16
   🤖 Chat/Instruct model ✓
   🏷️  ['chat', 'autotrain_compatible', 'text-generation-inference', 'endpoints_compatible']

📦 Qwen/Qwen3-4B-Base
   ⬇️  5,476,711 downloads, ❤️  50 likes
   📅 Created: 2025-04-28
   🏷️  ['autotrain_compatible', 'text-generation-inference', 'endpoints_compatible']

📦 Qwen/Qwen3-8B
   ⬇️  5,160,545 downloads, ❤️  544 likes
   📅 Created: 2025-04-27
   🏷️  ['autotrain_compatible', 'text-generation-inference', 'endpoints_compatible']

📦 Qwen/Qwen3-Embedding-0.6B
   ⬇️  3,311,960 downloads, ❤️  493 likes
   📅 Created: 2025-06-03
   🏷️  ['text-embeddings-inference', 'autotrain_compatible', 'text-generation-inference', '

In [7]:
from huggingface_hub import HfApi

api = HfApi()

# Get Qwen text-generation models
qwen_models = api.list_models(
    author="Qwen", 
    task="text-generation",
    limit=None,
    sort="downloads", 
    direction=-1
)

# Filter for likely chat/instruct models
good_qwen_models = []

for model in qwen_models:
    # Look for instruct/chat indicators
    is_instruct = any(keyword in model.id.lower() for keyword in [
        'instruct', 'chat', 'conversation'
    ])
    
    # Check if it has good download numbers and is public
    if is_instruct and not model.private and model.downloads > 100:
        good_qwen_models.append(model.id)

print(f"Best Qwen models for testing ({len(good_qwen_models)} found):")
for model_id in good_qwen_models[:8]:  # Top 8
    print(f"  - {model_id}")

# Test these models
if good_qwen_models:
    print(f"\nTesting top {min(5, len(good_qwen_models))} Qwen models...")
    result = check_model_supported(good_qwen_models[:5])
    print(f"Working Qwen models: {result['correct_models']}")

Best Qwen models for testing (151 found):
  - Qwen/Qwen2.5-14B-Instruct
  - Qwen/Qwen2.5-7B-Instruct
  - Qwen/Qwen2.5-1.5B-Instruct
  - Qwen/Qwen2.5-7B-Instruct-1M
  - Qwen/Qwen2.5-3B-Instruct
  - Qwen/Qwen2.5-0.5B-Instruct
  - Qwen/Qwen1.5-0.5B-Chat
  - Qwen/Qwen2-1.5B-Instruct

Testing top 5 Qwen models...
Testing 5 HuggingFace model(s)...
Testing: Qwen/Qwen2.5-14B-Instruct
   ❌ FAILED - Model not supported: Error code: 400 - {'error': {'message': "The requested model 'Qwen/Qwen2.5-14B-Instruct' is not supported by any provider you have enabled.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_supported'}}

Testing: Qwen/Qwen2.5-7B-Instruct
   ✅ SUPPORTED - Response: '2' (CORRECT ✓)

Testing: Qwen/Qwen2.5-1.5B-Instruct
   ❌ FAILED - Model not supported: Error code: 400 - {'error': {'message': "The requested model 'Qwen/Qwen2.5-1.5B-Instruct' is not supported by any provider you have enabled.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_no

In [8]:
from huggingface_hub import HfApi
from datetime import datetime

api = HfApi()

# Get Qwen text-generation models (get more to have a good selection)
print("Fetching Qwen text-generation models...")
qwen_models = api.list_models(
    author="Qwen", 
    task="text-generation",
    limit=None,  # Get all
    sort="lastModified",  # Sort by last modified instead of downloads
    direction=-1
)

# Filter for likely chat/instruct models with release dates
good_qwen_models = []

for model in qwen_models:
    # Look for instruct/chat indicators
    is_instruct = any(keyword in model.id.lower() for keyword in [
        'instruct', 'chat', 'conversation'
    ])
    
    # Check if it has good stats and is public
    if is_instruct and not model.private and model.downloads > 50:  # Lower threshold for newer models
        good_qwen_models.append({
            'id': model.id,
            'created_at': model.created_at,
            'downloads': model.downloads,
            'likes': model.likes,
            'tags': model.tags
        })

# Sort by creation date (newest first)
good_qwen_models.sort(key=lambda x: x['created_at'] if x['created_at'] else datetime.min, reverse=True)

print(f"\n🎯 Best Qwen instruct/chat models (sorted by release date, newest first):")
print("=" * 80)

# Show top 50
for i, model in enumerate(good_qwen_models[:50], 1):
    created_date = model['created_at'].strftime('%Y-%m-%d') if model['created_at'] else 'Unknown'
    
    print(f"{i:2d}. 📦 {model['id']}")
    print(f"    📅 Released: {created_date}")
    print(f"    ⬇️  Downloads: {model['downloads']:,} | ❤️  Likes: {model['likes']}")
    
    # Show model size info from model name
    size_indicators = ['0.5B', '1.5B', '3B', '7B', '14B', '32B', '72B', '110B']
    model_size = "Unknown size"
    for size in size_indicators:
        if size in model['id']:
            model_size = size
            break
    print(f"    📏 Size: {model_size}")
    
    # Show if it has inference tags
    if model['tags']:
        inference_tags = [tag for tag in model['tags'] if 
                         any(keyword in tag.lower() for keyword in 
                             ['inference', 'compatible', 'endpoints'])]
        if inference_tags:
            print(f"    🤖 Inference: {', '.join(inference_tags)}")
    
    print()

# Extract just the model IDs for testing
model_ids_for_testing = [model['id'] for model in good_qwen_models[:10]]  # Top 10 newest

print(f"🧪 Testing top 10 newest Qwen models...")
print("Models to test:", model_ids_for_testing)

# Test these models
if model_ids_for_testing:
    result = check_model_supported(model_ids_for_testing)
    
    print(f"\n✅ Working Qwen models ({result['correct_count']} out of {result['total_tested']}):")
    for model in result['correct_models']:
        # Find the release date for this model
        model_info = next((m for m in good_qwen_models if m['id'] == model), None)
        if model_info:
            release_date = model_info['created_at'].strftime('%Y-%m-%d') if model_info['created_at'] else 'Unknown'
            print(f"  - {model} (Released: {release_date})")
        else:
            print(f"  - {model}")

# Show summary by release month
print(f"\n📊 Release timeline summary:")
from collections import defaultdict

monthly_counts = defaultdict(int)
for model in good_qwen_models:
    if model['created_at']:
        month_key = model['created_at'].strftime('%Y-%m')
        monthly_counts[month_key] += 1

sorted_months = sorted(monthly_counts.items(), reverse=True)
for month, count in sorted_months[:12]:  # Last 12 months
    print(f"  {month}: {count} models")

Fetching Qwen text-generation models...

🎯 Best Qwen instruct/chat models (sorted by release date, newest first):
 1. 📦 Qwen/Qwen3-4B-Instruct-2507-FP8
    📅 Released: 2025-08-06
    ⬇️  Downloads: 5,219 | ❤️  Likes: 22
    📏 Size: Unknown size
    🤖 Inference: autotrain_compatible, text-generation-inference, endpoints_compatible

 2. 📦 Qwen/Qwen3-4B-Instruct-2507
    📅 Released: 2025-08-05
    ⬇️  Downloads: 140,077 | ❤️  Likes: 223
    📏 Size: Unknown size
    🤖 Inference: autotrain_compatible, text-generation-inference, endpoints_compatible

 3. 📦 Qwen/Qwen3-Coder-30B-A3B-Instruct-FP8
    📅 Released: 2025-07-31
    ⬇️  Downloads: 38,028 | ❤️  Likes: 55
    📏 Size: 3B
    🤖 Inference: autotrain_compatible, endpoints_compatible

 4. 📦 Qwen/Qwen3-Coder-30B-A3B-Instruct
    📅 Released: 2025-07-31
    ⬇️  Downloads: 202,919 | ❤️  Likes: 481
    📏 Size: 3B
    🤖 Inference: autotrain_compatible, endpoints_compatible

 5. 📦 Qwen/Qwen3-30B-A3B-Instruct-2507-FP8
    📅 Released: 2025-07-28
   

In [12]:
from huggingface_hub import HfApi
from datetime import datetime
import pandas as pd
import re

api = HfApi()

def extract_parameters(model_name):
    """Extract number of parameters from model name and convert to integer"""
    # Look for patterns like 0.5B, 1.5B, 3B, 7B, 14B, 32B, 72B, 110B
    pattern = r'(\d+(?:\.\d+)?)[Bb]'
    match = re.search(pattern, model_name)
    
    if match:
        size_str = match.group(1)
        size_float = float(size_str)
        # Convert to actual number (B = billion)
        return int(size_float * 1_000_000_000)
    
    return None  # Unknown size

# Get Qwen text-generation models
print("Fetching Qwen text-generation models...")
qwen_models = api.list_models(
    author="Qwen", 
    task="text-generation",
    limit=None,
    sort="lastModified",
    direction=-1
)

# Filter for likely chat/instruct models
good_qwen_models = []

for model in qwen_models:
    # Look for instruct/chat indicators
    is_instruct = any(keyword in model.id.lower() for keyword in [
        'instruct', 'chat', 'conversation'
    ])
    
    # Check if it has good stats and is public
    if is_instruct and not model.private and model.downloads > 5:
        good_qwen_models.append({
            'id': model.id,
            'created_at': model.created_at,
            'downloads': model.downloads,
            'likes': model.likes,
            'tags': model.tags
        })

# Sort by creation date (newest first)
good_qwen_models.sort(key=lambda x: x['created_at'] if x['created_at'] else datetime.min, reverse=True)

# Take top 100
top_100_models = good_qwen_models # good_qwen_models[:100]

print(f"Preparing to test top {len(top_100_models)} Qwen models...")

# Test all models for support
model_ids_for_testing = [model['id'] for model in top_100_models]
print("Testing models for HuggingFace support...")

# Test the models (this might take a while)
support_result = check_model_supported(model_ids_for_testing)

# Create DataFrame
df_data = []

for model in top_100_models:
    model_name = model['id']
    
    # Extract just the model name (remove "Qwen/" prefix)
    display_name = model_name.split('/')[-1]
    
    # Create URL
    url = f"https://huggingface.co/{model_name}"
    
    # Extract parameters
    parameters = extract_parameters(model_name)
    
    # Format release date
    released = model['created_at'].strftime('%Y-%m-%d') if model['created_at'] else None
    
    # Downloads
    downloads = model['downloads']
    
    # Check if supported
    supported = model_name in support_result['correct_models']
    
    df_data.append({
        'model_name': display_name,
        'url': url,
        'parameters': parameters,
        'released': released,
        'downloads': downloads,
        'supported': supported
    })

# Create DataFrame
df = pd.DataFrame(df_data)

# Sort by release date (newest first) and then by downloads
df['released_dt'] = pd.to_datetime(df['released'])
df = df.sort_values(['released_dt', 'downloads'], ascending=[False, False])

# Drop the helper column
df = df.drop('released_dt', axis=1)

print(f"\n🎯 Qwen Instruct/Chat Models Summary:")
print(f"Total models: {len(df)}")
print(f"Supported models: {df['supported'].sum()}")
print(f"Models with known parameters: {df['parameters'].notna().sum()}")

# Display the DataFrame
print(f"\nTop 100 Qwen Models (sorted by release date, newest first):")
print("=" * 100)

# Format the display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Show the dataframe
display(df)

# Additional summary statistics
print(f"\n📊 Summary Statistics:")
print(f"✅ Supported models: {df['supported'].sum()} out of {len(df)} ({df['supported'].mean()*100:.1f}%)")

if df['parameters'].notna().any():
    print(f"📏 Parameter sizes:")
    param_summary = df[df['parameters'].notna()]['parameters'].value_counts().sort_index()
    for params, count in param_summary.items():
        size_b = params / 1_000_000_000
        print(f"   {size_b:g}B: {count} models")

print(f"📅 Release date range: {df['released'].min()} to {df['released'].max()}")
print(f"⬇️  Download range: {df['downloads'].min():,} to {df['downloads'].max():,}")

# Show working models
working_models = df[df['supported'] == True]
if not working_models.empty:
    print(f"\n✅ Working Models ({len(working_models)}):")
    for _, row in working_models.iterrows():
        params_str = f"{row['parameters']/1_000_000_000:g}B" if pd.notna(row['parameters']) else "Unknown"
        print(f"   - {row['model_name']} ({params_str}, {row['downloads']:,} downloads)")

# Return the dataframe for further analysis
df

Fetching Qwen text-generation models...
Preparing to test top 167 Qwen models...
Testing models for HuggingFace support...
Testing 167 HuggingFace model(s)...
Testing: Qwen/Qwen3-4B-Instruct-2507-FP8
   ❌ FAILED - Model not supported: Error code: 400 - {'error': {'message': "The requested model 'Qwen/Qwen3-4B-Instruct-2507-FP8' is not supported by any provider you have enabled.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_supported'}}

Testing: Qwen/Qwen3-4B-Instruct-2507
   ✅ SUPPORTED - Response: '2' (CORRECT ✓)

Testing: Qwen/Qwen3-Coder-30B-A3B-Instruct-FP8
   ❌ FAILED - Model not supported: Error code: 400 - {'error': {'message': "The requested model 'Qwen/Qwen3-Coder-30B-A3B-Instruct-FP8' is not supported by any provider you have enabled.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_supported'}}

Testing: Qwen/Qwen3-Coder-30B-A3B-Instruct
   ✅ SUPPORTED - Response: '2' (CORRECT ✓)

Testing: Qwen/Qwen3-30B-A3B-Instruct-2507-FP8

,model_name,url,parameters,released,downloads,supported
0,Qwen3-4B-Instruct-2507-FP8,https://huggingface.co/Qwen/Qwen3-4B-Instruct-...,4.000000e+09,2025-08-06,5219,False
1,Qwen3-4B-Instruct-2507,https://huggingface.co/Qwen/Qwen3-4B-Instruct-...,4.000000e+09,2025-08-05,140077,True
3,Qwen3-Coder-30B-A3B-Instruct,https://huggingface.co/Qwen/Qwen3-Coder-30B-A3...,3.000000e+10,2025-07-31,202919,True
2,Qwen3-Coder-30B-A3B-Instruct-FP8,https://huggingface.co/Qwen/Qwen3-Coder-30B-A3...,3.000000e+10,2025-07-31,38028,False
5,Qwen3-30B-A3B-Instruct-2507,https://huggingface.co/Qwen/Qwen3-30B-A3B-Inst...,3.000000e+10,2025-07-28,446351,False
...,...,...,...,...,...,...
162,Qwen-14B-Chat-Int4,https://huggingface.co/Qwen/Qwen-14B-Chat-Int4,1.400000e+10,2023-09-24,912,False
163,Qwen-VL-Chat-Int4,https://huggingface.co/Qwen/Qwen-VL-Chat-Int4,NaN,2023-08-31,2651,False
165,Qwen-VL-Chat,https://huggingface.co/Qwen/Qwen-VL-Chat,NaN,2023-08-20,70651,False
164,Qwen-7B-Chat-Int4,https://huggingface.co/Qwen/Qwen-7B-Chat-Int4,7.000000e+09,2023-08-20,1658,False



📊 Summary Statistics:
✅ Supported models: 12 out of 167 (7.2%)
📏 Parameter sizes:
   0.5B: 21 models
   1.5B: 18 models
   1.8B: 5 models
   2.7B: 2 models
   3B: 10 models
   4B: 7 models
   7B: 30 models
   8B: 3 models
   14B: 19 models
   30B: 4 models
   32B: 14 models
   57B: 3 models
   72B: 20 models
   110B: 4 models
   235B: 2 models
   480B: 2 models
📅 Release date range: 2023-08-03 to 2025-08-06
⬇️  Download range: 13 to 12,195,236

✅ Working Models (12):
   - Qwen3-4B-Instruct-2507 (4B, 140,077 downloads)
   - Qwen3-Coder-30B-A3B-Instruct (30B, 202,919 downloads)
   - Qwen3-Coder-480B-A35B-Instruct-FP8 (480B, 112,368 downloads)
   - Qwen3-Coder-480B-A35B-Instruct (480B, 59,794 downloads)
   - Qwen3-235B-A22B-Instruct-2507 (235B, 71,540 downloads)
   - Qwen2.5-Coder-3B-Instruct (3B, 150,862 downloads)
   - Qwen2.5-Coder-32B-Instruct (32B, 73,622 downloads)
   - Qwen2.5-32B-Instruct (32B, 346,916 downloads)
   - Qwen2.5-Coder-7B-Instruct (7B, 244,138 downloads)
   - Qwen2.5

,model_name,url,parameters,released,downloads,supported
0,Qwen3-4B-Instruct-2507-FP8,https://huggingface.co/Qwen/Qwen3-4B-Instruct-...,4.000000e+09,2025-08-06,5219,False
1,Qwen3-4B-Instruct-2507,https://huggingface.co/Qwen/Qwen3-4B-Instruct-...,4.000000e+09,2025-08-05,140077,True
3,Qwen3-Coder-30B-A3B-Instruct,https://huggingface.co/Qwen/Qwen3-Coder-30B-A3...,3.000000e+10,2025-07-31,202919,True
2,Qwen3-Coder-30B-A3B-Instruct-FP8,https://huggingface.co/Qwen/Qwen3-Coder-30B-A3...,3.000000e+10,2025-07-31,38028,False
5,Qwen3-30B-A3B-Instruct-2507,https://huggingface.co/Qwen/Qwen3-30B-A3B-Inst...,3.000000e+10,2025-07-28,446351,False
...,...,...,...,...,...,...
162,Qwen-14B-Chat-Int4,https://huggingface.co/Qwen/Qwen-14B-Chat-Int4,1.400000e+10,2023-09-24,912,False
163,Qwen-VL-Chat-Int4,https://huggingface.co/Qwen/Qwen-VL-Chat-Int4,NaN,2023-08-31,2651,False
165,Qwen-VL-Chat,https://huggingface.co/Qwen/Qwen-VL-Chat,NaN,2023-08-20,70651,False
164,Qwen-7B-Chat-Int4,https://huggingface.co/Qwen/Qwen-7B-Chat-Int4,7.000000e+09,2023-08-20,1658,False


In [13]:
working_df = df[df['supported'] == True]
working_df.to_csv('qwen_models_analysis.csv', index=False)

In [41]:
first_model = list(models)[0]
print("Model object type:", type(first_model))
print("\nAvailable attributes:")
for attr in dir(first_model):
    if not attr.startswith('_'):  # Skip private attributes
        try:
            value = getattr(first_model, attr)
            if not callable(value):  # Skip methods
                print(f"  {attr}: {type(value)} = {value}")
        except:
            print(f"  {attr}: <error accessing>")

IndexError: list index out of range

In [48]:
from huggingface_hub import HfApi

api = HfApi()

# Get text-generation models
models = api.list_models(
    task="text-generation",
    limit=1000,
    sort="downloads", 
    direction=-1
)

# Filter for likely chat/instruct models
good_models = []

for model in models:
    # Check if it's a chat/instruct model
    is_chat_model = any(keyword in model.id.lower() for keyword in [
        'instruct', 'chat', 'conversation'
    ])
    
    # Check tags for chat/instruct indicators
    has_chat_tags = False
    if model.tags:
        has_chat_tags = any(keyword in ' '.join(model.tags).lower() for keyword in [
            'instruct', 'chat', 'conversation', 'text-generation-inference'
        ])
    
    # Filter criteria
    if (is_chat_model or has_chat_tags) and \
       model.library_name == 'transformers' and \
       not model.private and \
       model.downloads > 1000:  # Has some usage
        
        good_models.append({
            'id': model.id,
            'downloads': model.downloads,
            'likes': model.likes,
            'tags': model.tags
        })

# Sort by downloads and show top candidates
good_models.sort(key=lambda x: x['downloads'], reverse=True)

print(f"Found {len(good_models)} promising chat/instruct models:")
print("=" * 60)

for model in good_models[:10]:
    print(f"📦 {model['id']}")
    print(f"   ⬇️  {model['downloads']:,} downloads, ❤️  {model['likes']} likes")
    
    # Show relevant tags
    relevant_tags = [tag for tag in model['tags'] if 
                    any(keyword in tag.lower() for keyword in 
                        ['instruct', 'chat', 'inference', 'compatible'])]
    if relevant_tags:
        print(f"   🏷️  {relevant_tags}")
    print()

Found 756 promising chat/instruct models:
📦 meta-llama/Llama-3.1-8B-Instruct
   ⬇️  13,028,195 downloads, ❤️  4489 likes
   🏷️  ['autotrain_compatible', 'text-generation-inference', 'endpoints_compatible']

📦 Qwen/Qwen2.5-14B-Instruct
   ⬇️  12,195,236 downloads, ❤️  264 likes
   🏷️  ['chat', 'autotrain_compatible', 'text-generation-inference', 'endpoints_compatible']

📦 openai-community/gpt2
   ⬇️  10,619,980 downloads, ❤️  2899 likes
   🏷️  ['autotrain_compatible', 'text-generation-inference', 'endpoints_compatible']

📦 Qwen/Qwen2.5-7B-Instruct
   ⬇️  9,128,290 downloads, ❤️  760 likes
   🏷️  ['chat', 'autotrain_compatible', 'text-generation-inference', 'endpoints_compatible']

📦 facebook/opt-125m
   ⬇️  7,325,030 downloads, ❤️  211 likes
   🏷️  ['autotrain_compatible', 'text-generation-inference']

📦 Qwen/Qwen3-4B-Base
   ⬇️  5,476,711 downloads, ❤️  50 likes
   🏷️  ['autotrain_compatible', 'text-generation-inference', 'endpoints_compatible']

📦 Qwen/Qwen3-8B
   ⬇️  5,160,545 downlo

In [49]:
# Extract just the model IDs from your filtered list
model_ids_to_test = [model['id'] for model in good_models[:100]]

print("Testing promising models:")
print(model_ids_to_test)

# Test them with your function
if model_ids_to_test:
    result = check_model_supported(model_ids_to_test)
    print(f"\n🎯 Working models: {result['correct_models']}")

Testing promising models:
['meta-llama/Llama-3.1-8B-Instruct', 'Qwen/Qwen2.5-14B-Instruct', 'openai-community/gpt2', 'Qwen/Qwen2.5-7B-Instruct', 'facebook/opt-125m', 'Qwen/Qwen3-4B-Base', 'Qwen/Qwen3-8B', 'dphn/dolphin-2.9.1-yi-1.5-34b', 'openai/gpt-oss-20b', 'context-labs/meta-llama-Llama-3.2-3B-Instruct-FP16', 'Gensyn/Qwen2.5-0.5B-Instruct', 'meta-llama/Llama-3.2-1B-Instruct', 'distilbert/distilgpt2', 'Qwen/Qwen2.5-1.5B-Instruct', 'Qwen/Qwen3-0.6B', 'openai-community/gpt2-large', 'google/gemma-3-1b-it', 'meta-llama/Llama-3.2-1B', 'trl-internal-testing/tiny-Qwen2ForCausalLM-2.5', 'Qwen/Qwen3-8B-Base', 'deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', 'petals-team/StableBeluga2', 'meta-llama/Llama-3.2-3B-Instruct', 'Qwen/Qwen2.5-7B-Instruct-1M', 'TinyLlama/TinyLlama-1.1B-Chat-v1.0', 'AIDC-AI/Ovis2-4B', 'Qwen/Qwen3-1.7B', 'Qwen/Qwen3-0.6B-Base', 'Qwen/Qwen2.5-3B-Instruct', 'Qwen/Qwen3-4B', 'google-t5/t5-3b', 'microsoft/DialoGPT-medium', 'Qwen/Qwen2.5-0.5B-Instruct', 'deepseek-ai/DeepSeek-R1-Di

In [50]:
from huggingface_hub import HfApi
import requests

# First, let's see what providers are actually available
def check_available_providers():
    """Check what inference providers are actually available in the API"""
    url = "https://huggingface.co/api/models"
    params = {"limit": 100}
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        models = response.json()
        
        # Check what provider-related fields exist
        sample_model = models[0] if models else {}
        print("Sample model fields:", list(sample_model.keys()))
        
        # Look for any provider-related info
        for model in models[:99]:
            if 'inference' in str(model).lower() or 'provider' in str(model).lower():
                print(f"Model with provider info: {model}")
                break
    
check_available_providers()

Sample model fields: ['_id', 'id', 'likes', 'trendingScore', 'private', 'downloads', 'tags', 'pipeline_tag', 'library_name', 'createdAt', 'modelId']
Model with provider info: {'_id': '689252773b8900ddb9116aed', 'id': 'google/gemma-3-270m', 'likes': 469, 'trendingScore': 469, 'private': False, 'downloads': 6487, 'tags': ['transformers', 'safetensors', 'gemma3_text', 'text-generation', 'gemma3', 'gemma', 'google', 'arxiv:2503.19786', 'arxiv:1905.07830', 'arxiv:1905.10044', 'arxiv:1911.11641', 'arxiv:1705.03551', 'arxiv:1911.01547', 'arxiv:1907.10641', 'arxiv:2311.07911', 'arxiv:2311.12022', 'arxiv:2411.04368', 'arxiv:1904.09728', 'arxiv:1903.00161', 'arxiv:2009.03300', 'arxiv:2304.06364', 'arxiv:2103.03874', 'arxiv:2110.14168', 'arxiv:2108.07732', 'arxiv:2107.03374', 'arxiv:2403.07974', 'arxiv:2305.03111', 'arxiv:2405.04520', 'arxiv:2210.03057', 'arxiv:2106.03193', 'arxiv:1910.11856', 'arxiv:2502.12404', 'arxiv:2502.21228', 'arxiv:2404.16816', 'arxiv:2104.12756', 'arxiv:2311.16502', 'arx